# 🌲 Мониторинг экосистемы через IT-решения

## Комплексная система анализа лесной экосистемы

**Автор:** Проект для экологического мониторинга  
**Дата:** Февраль 2026  
**Версия:** 1.0.0

---

## 📋 Содержание

1. [Введение и постановка задачи](#1)
2. [Установка и импорт библиотек](#2)
3. [Модуль анализа видео (YOLOv8)](#3)
4. [Модуль анализа аудио (YAMNet)](#4)
5. [Визуализация данных](#5)
6. [Система оповещений](#6)
7. [Комплексный анализ и отчёты](#7)
8. [Выводы](#8)

<a id='1'></a>
## 1. 🎯 Введение и постановка задачи

### Проблема

Окружающая среда подвергается постоянным изменениям под влиянием естественных факторов и человеческой деятельности. Экологам необходимы надёжные данные для:

- 📊 Мониторинга состояния экосистем
- 🔍 Выявления изменений в популяциях животных
- 🚨 Раннего обнаружения экологических угроз
- 📈 Принятия обоснованных решений по охране природы

**Традиционные методы** сбора данных:
- ❌ Трудоёмки и затратны
- ❌ Подвержены человеческим ошибкам
- ❌ Не обеспечивают непрерывный мониторинг
- ❌ Требуют постоянного присутствия специалистов

### Решение

**Автоматизированная система мониторинга** с использованием:

1. **Компьютерное зрение (YOLOv8)** - обнаружение и классификация животных на видео
2. **Анализ аудио (YAMNet)** - распознавание голосов птиц и звуков природы
3. **Машинное обучение** - выявление трендов и аномалий
4. **Визуализация данных** - наглядное представление результатов
5. **Система оповещений** - автоматические уведомления о критических событиях

### Целевые пользователи

- 🌿 Экологи и биологи
- 🏞️ Сотрудники национальных парков
- 🔬 Научные исследователи
- 🏛️ Природоохранные организации
- 📋 Органы государственного контроля

<a id='2'></a>
## 2. 📦 Установка и импорт библиотек

### Установка зависимостей

Если вы ещё не установили необходимые библиотеки, раскомментируйте и выполните следующую ячейку:

In [8]:
# Установка необходимых библиотек
# Раскомментируйте при первом запуске

# !pip install ultralytics opencv-python librosa tensorflow tensorflow-hub matplotlib seaborn plotly pandas numpy scipy

### Импорт библиотек

In [9]:
# Стандартные библиотеки Python
import os
import sys
import warnings
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter

# Подавление предупреждений для чистоты вывода
warnings.filterwarnings('ignore')

# Работа с данными
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Настройка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Обработка видео и изображений
import cv2
from PIL import Image

# YOLO для детекции объектов
from ultralytics import YOLO

# TensorFlow для аудио анализа
import tensorflow as tf
import tensorflow_hub as hub

# Обработка аудио
import librosa
import librosa.display
import soundfile as sf
from scipy import signal

# Утилиты
from tqdm.notebook import tqdm
from IPython.display import display, HTML, Video, Audio, Image as IPImage

print("✅ Все библиотеки успешно импортированы!")
print(f"📅 Дата запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Версия Python: {sys.version.split()[0]}")
print(f"🤖 TensorFlow версия: {tf.__version__}")

✅ Все библиотеки успешно импортированы!
📅 Дата запуска: 2026-02-10 00:07:38
🐍 Версия Python: 3.10.11
🤖 TensorFlow версия: 2.13.0


### Создание структуры директорий

In [10]:
# Создание необходимых директорий для проекта
directories = {
    'data': 'Основная директория для данных',
    'data/videos': 'Видеофайлы для анализа',
    'data/audio': 'Аудиозаписи',
    'data/images': 'Изображения',
    'data/results': 'Результаты анализа',
    'data/results/videos': 'Обработанные видео',
    'data/results/reports': 'Отчёты',
    'models': 'Предобученные модели'
}

for dir_path, description in directories.items():
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    print(f"📁 {dir_path:30s} - {description}")

print("\n✅ Структура директорий создана!")

📁 data                           - Основная директория для данных
📁 data/videos                    - Видеофайлы для анализа
📁 data/audio                     - Аудиозаписи
📁 data/images                    - Изображения
📁 data/results                   - Результаты анализа
📁 data/results/videos            - Обработанные видео
📁 data/results/reports           - Отчёты
📁 models                         - Предобученные модели

✅ Структура директорий создана!


<a id='3'></a>
## 3. 🎥 Модуль анализа видео (YOLOv8)

### О модели YOLOv8

**YOLO (You Only Look Once)** - это семейство моделей для детекции объектов в реальном времени.

**YOLOv8** - последняя версия от Ultralytics:
- ✅ Предобучена на датасете COCO (80 классов объектов)
- ✅ Высокая точность и скорость работы
- ✅ Поддержка различных размеров моделей (nano, small, medium, large, xlarge)
- ✅ Простой API для использования

**Классы животных в COCO:**
- 🐦 bird (птица)
- 🐻 bear (медведь)
- 🐘 elephant (слон)
- 🦒 giraffe (жираф)
- 🦓 zebra (зебра)
- И многие другие...

### 3.1 Загрузка модели YOLOv8

In [11]:
# Загрузка предобученной модели YOLOv8
# При первом запуске модель автоматически скачается из репозитория Ultralytics

print("📥 Загрузка модели YOLOv8...")

# Используем nano версию для быстрой работы
# Доступные версии: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
# n = nano (самая быстрая, но менее точная)
# x = extra large (самая точная, но медленная)
model_yolo = YOLO('yolov8n.pt')

print("✅ Модель YOLOv8 успешно загружена!")
print(f"📊 Модель обучена на {len(model_yolo.names)} классах объектов")

# Классы животных из COCO dataset
ANIMAL_CLASSES = {
    14: 'bird',      # Птица
    15: 'cat',       # Кошка
    16: 'dog',       # Собака
    17: 'horse',     # Лошадь
    18: 'sheep',     # Овца
    19: 'cow',       # Корова
    20: 'elephant',  # Слон
    21: 'bear',      # Медведь
    22: 'zebra',     # Зебра
    23: 'giraffe',   # Жираф
}

print("\n🐾 Отслеживаемые классы животных:")
for class_id, class_name in ANIMAL_CLASSES.items():
    print(f"  {class_id:2d}: {class_name}")

📥 Загрузка модели YOLOv8...
✅ Модель YOLOv8 успешно загружена!
📊 Модель обучена на 80 классах объектов

🐾 Отслеживаемые классы животных:
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe


### 3.2 Класс для анализа видео

In [12]:
class VideoAnalyzer:
    """
    Класс для анализа видео с использованием YOLOv8
    
    Функциональность:
    - Обнаружение животных на видео
    - Подсчёт уникальных особей
    - Отслеживание траекторий движения
    - Создание аннотированного видео
    - Формирование статистики
    """
    
    def __init__(self, model, confidence_threshold=0.25):
        """
        Инициализация анализатора
        
        Args:
            model: Загруженная модель YOLO
            confidence_threshold: Минимальный порог уверенности для детекции
        """
        self.model = model
        self.confidence_threshold = confidence_threshold
        self.detections = []  # История всех детекций
        self.statistics = defaultdict(int)  # Статистика по классам
        
    def process_frame(self, frame):
        """
        Обработка одного кадра видео
        
        Args:
            frame: Кадр видео (numpy array)
            
        Returns:
            frame: Аннотированный кадр
            detections: Список детекций на кадре
        """
        # Запуск детекции на кадре
        results = self.model(frame, conf=self.confidence_threshold, verbose=False)
        
        detections = []
        
        # Обработка результатов детекции
        for result in results:
            boxes = result.boxes
            
            for box in boxes:
                # Получение параметров детекции
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                bbox = box.xyxy[0].cpu().numpy()
                
                # Фильтрация только животных
                if class_id in ANIMAL_CLASSES:
                    class_name = ANIMAL_CLASSES[class_id]
                    
                    # Сохранение детекции
                    detection = {
                        'class_id': class_id,
                        'class_name': class_name,
                        'confidence': confidence,
                        'bbox': bbox
                    }
                    detections.append(detection)
                    self.statistics[class_name] += 1
                    
                    # Отрисовка bounding box
                    x1, y1, x2, y2 = map(int, bbox)
                    
                    # Цвет в зависимости от класса
                    color = (0, 255, 0) if class_name == 'bird' else (255, 0, 0)
                    
                    # Рисование прямоугольника
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    
                    # Подпись с классом и уверенностью
                    label = f"{class_name} {confidence:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        return frame, detections
    
    def analyze_video(self, video_path, output_path=None, fps_sample=2):
        """
        Анализ всего видеофайла
        
        Args:
            video_path: Путь к входному видео
            output_path: Путь для сохранения результата (опционально)
            fps_sample: Обрабатывать каждый N-й кадр (для ускорения)
            
        Returns:
            results: Словарь с результатами анализа
        """
        print(f"🎬 Начало анализа видео: {video_path}")
        
        # Открытие видеофайла
        cap = cv2.VideoCapture(str(video_path))
        
        if not cap.isOpened():
            raise ValueError(f"Не удалось открыть видео: {video_path}")
        
        # Получение параметров видео
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0
        
        print(f"📊 Параметры видео:")
        print(f"   Разрешение: {width}x{height}")
        print(f"   FPS: {fps}")
        print(f"   Кадров: {frame_count}")
        print(f"   Длительность: {duration:.1f} сек")
        
        # Инициализация записи выходного видео
        out = None
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
        
        # Сброс статистики
        self.statistics = defaultdict(int)
        self.detections = []
        
        frame_idx = 0
        processed_frames = 0
        
        # Обработка видео с прогресс-баром
        pbar = tqdm(total=frame_count, desc="Обработка кадров")
        
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                break
            
            # Обработка только каждого N-го кадра для ускорения
            if frame_idx % fps_sample == 0:
                # Обработка кадра
                annotated_frame, detections = self.process_frame(frame)
                
                # Сохранение детекций с временной меткой
                timestamp = frame_idx / fps if fps > 0 else 0
                for det in detections:
                    det['timestamp'] = timestamp
                    det['frame'] = frame_idx
                    self.detections.append(det)
                
                # Запись обработанного кадра
                if out is not None:
                    out.write(annotated_frame)
                
                processed_frames += 1
            
            frame_idx += 1
            pbar.update(1)
        
        pbar.close()
        
        # Освобождение ресурсов
        cap.release()
        if out is not None:
            out.release()
        
        # Формирование результатов
        results = {
            'video_path': str(video_path),
            'output_path': str(output_path) if output_path else None,
            'duration': duration,
            'total_frames': frame_count,
            'processed_frames': processed_frames,
            'fps': fps,
            'resolution': (width, height),
            'detections': self.detections,
            'statistics': dict(self.statistics),
            'unique_species': len(self.statistics)
        }
        
        print(f"\n✅ Анализ завершён!")
        print(f"📊 Обработано кадров: {processed_frames}/{frame_count}")
        print(f"🐾 Всего детекций: {len(self.detections)}")
        print(f"🦋 Уникальных видов: {results['unique_species']}")
        
        return results
    
    def get_statistics_df(self):
        """
        Получение статистики в виде DataFrame
        
        Returns:
            DataFrame с детекциями
        """
        if not self.detections:
            return pd.DataFrame()
        
        df = pd.DataFrame(self.detections)
        return df

print("✅ Класс VideoAnalyzer создан!")

✅ Класс VideoAnalyzer создан!


### 3.3 Создание демо-видео для тестирования

In [13]:
def create_demo_video(output_path='data/videos/demo_wildlife.mp4', duration=10, fps=30):
    """
    Создание демонстрационного видео с синтетическими данными
    
    Args:
        output_path: Путь для сохранения видео
        duration: Длительность в секундах
        fps: Кадры в секунду
    """
    print("🎨 Создание демонстрационного видео...")
    
    width, height = 1280, 720
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    total_frames = duration * fps
    
    for frame_num in range(total_frames):
        # Создание зелёного фона (имитация леса)
        frame = np.ones((height, width, 3), dtype=np.uint8) * [34, 139, 34]  # Зелёный
        
        # Добавление «земли»
        cv2.rectangle(frame, (0, height - 150), (width, height), (101, 67, 33), -1)
        
        # Добавление «неба»
        cv2.rectangle(frame, (0, 0), (width, 150), (135, 206, 235), -1)
        
        # Добавление текста
        t = frame_num / fps
        text = f"Demo Wildlife Video - Time: {t:.1f}s"
        cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # Добавление движущихся "животных" (круги)
        # Симуляция птицы
        bird_x = int(200 + 10 * frame_num)
        bird_y = int(200 + 50 * np.sin(frame_num / 10))
        cv2.circle(frame, (bird_x % width, bird_y), 20, (255, 255, 0), -1)
        cv2.putText(frame, "Bird", (bird_x % width - 20, bird_y - 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Симуляция оленя
        deer_x = int(800 - 5 * frame_num)
        deer_y = height - 200
        cv2.circle(frame, (deer_x % width, deer_y), 30, (165, 42, 42), -1)
        cv2.putText(frame, "Deer", (deer_x % width - 20, deer_y - 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        out.write(frame)
    
    out.release()
    print(f"✅ Демо-видео создано: {output_path}")
    print(f"   Длительность: {duration}с, FPS: {fps}, Кадров: {total_frames}")

# Создание демо-видео
demo_video_path = 'data/videos/demo_wildlife.mp4'
if not Path(demo_video_path).exists():
    create_demo_video(demo_video_path, duration=15, fps=30)
else:
    print(f"ℹ️  Демо-видео уже существует: {demo_video_path}")

ℹ️  Демо-видео уже существует: data/videos/demo_wildlife.mp4


### 3.4 Запуск анализа видео

In [16]:
# Инициализация анализатора
video_analyzer = VideoAnalyzer(model_yolo, confidence_threshold=0.3)

# Пути к файлам
input_video = 'data/videos/white_bears.mp4'
output_video = 'data/results/videos/analyzed_wildlife.mp4'

# Запуск анализа
video_results = video_analyzer.analyze_video(
    video_path=input_video,
    output_path=output_video,
    fps_sample=1  # Обрабатывать каждый кадр
)

# Вывод результатов
print("\n" + "="*60)
print("📊 РЕЗУЛЬТАТЫ АНАЛИЗА ВИДЕО")
print("="*60)
print(f"\n📹 Видео: {video_results['video_path']}")
print(f"⏱️  Длительность: {video_results['duration']:.1f} сек")
print(f"🎞️  Всего кадров: {video_results['total_frames']}")
print(f"✅ Обработано: {video_results['processed_frames']}")
print(f"\n🐾 Статистика детекций:")
for species, count in sorted(video_results['statistics'].items(), key=lambda x: x[1], reverse=True):
    print(f"   {species:15s}: {count:4d} детекций")
print(f"\n🦋 Уникальных видов: {video_results['unique_species']}")
print(f"📊 Всего детекций: {len(video_results['detections'])}")

🎬 Начало анализа видео: data/videos/white_bears.mp4
📊 Параметры видео:
   Разрешение: 1920x1080
   FPS: 30
   Кадров: 530
   Длительность: 17.7 сек


Обработка кадров:   0%|          | 0/530 [00:00<?, ?it/s]


✅ Анализ завершён!
📊 Обработано кадров: 530/530
🐾 Всего детекций: 484
🦋 Уникальных видов: 6

📊 РЕЗУЛЬТАТЫ АНАЛИЗА ВИДЕО

📹 Видео: data/videos/white_bears.mp4
⏱️  Длительность: 17.7 сек
🎞️  Всего кадров: 530
✅ Обработано: 530

🐾 Статистика детекций:
   bear           :  405 детекций
   dog            :   42 детекций
   elephant       :   20 детекций
   cow            :    7 детекций
   cat            :    7 детекций
   sheep          :    3 детекций

🦋 Уникальных видов: 6
📊 Всего детекций: 484
